In [1]:
import numpy as np
import scipy 
import scipy.sparse as sparse
import no_J_explicit_fixed_pump as api
import matplotlib.pyplot as plt
import cvxpy as cp

np.random.seed(0)

In [2]:
# Initialize parameters and SDR constraint
N_omega = 11
omega = np.linspace(-2, 2, N_omega)
N_z = 2
z = np.linspace(0, 6*10**-3, N_z)
delta_z = np.abs(z[1] - z[0])
green_fs = api.get_green_f(omega,z)
projection = np.zeros((N_omega, N_omega))
projections = []
sdr_def_constr = []
sdr_cst = []
for i in range(N_omega):
    for j in range(N_omega):
        proj_copy = projection.copy()
        proj_copy[i, j] = 1
        projections.append(sparse.csc_matrix(proj_copy))
        sdr_def_constr.append(api.sdr_def_constr(N_omega, N_z, sparse.csc_matrix(proj_copy)))
        if i == j:
            sdr_cst.append(2.)
        else:
            sdr_cst.append(0.)

In [3]:
beta_vec = np.exp(-(np.linspace(omega[0], omega[-1], 2*N_omega - 1)**2)/0.4)#np.random.random(2*N_omega - 1)#
rand_herm = np.random.random((N_omega, N_omega)) + 1.j*np.random.random((N_omega, N_omega))
rand_unitary = scipy.linalg.eig(rand_herm + rand_herm.conj().T)[1]
beta = scipy.linalg.hankel(beta_vec[:N_omega], beta_vec[N_omega - 1:])
beta_weight = np.sqrt(np.trace(beta.conj().T@beta))
new_beta = beta/beta_weight
delta_k = 1.j*np.diag(omega)
Q_plus = delta_k + beta
Q_minus = delta_k - beta
W_plus = [scipy.linalg.expm(Q_plus*z[i]) for i in range(N_z)]
W_minus = [scipy.linalg.expm(Q_minus*z[i]) for i in range(N_z)]
J = 0.25*(W_plus[-1]@W_plus[-1].conj().T + W_minus[-1]@W_minus[-1].conj().T - 2*np.eye(N_omega))
n = 0.25*np.trace((W_plus[-1] - W_minus[-1]).conj().T@(W_plus[-1] - W_minus[-1]))
W_plus = [(1/np.sqrt(n))*scipy.linalg.expm(Q_plus*z[i]) for i in range(N_z)]
W_minus = [(1/np.sqrt(n))*scipy.linalg.expm(Q_minus*z[i]) for i in range(N_z)]
X = np.vstack([rand_unitary] + W_plus + W_minus)
Y = np.vstack([rand_unitary] + W_plus + W_minus + [np.eye(N_omega)])


In [4]:
# Create constraint matrices
constr_sympl = []
constr_dyn = []
constr_unitary = []
photon_end = api.photon_nbr_constr(N_omega, N_z, n)
photon_nbr_prev = api.photon_nbr_prev_points(N_omega, N_z)
for i in range(len(projections)):
    sympl_real, sympl_imag = api.sympl_constr_sdr(N_omega, N_z, projections[i], n)
    dyn_real_plus, dyn_imag_plus, dyn_real_minus, dyn_imag_minus = api.get_dynamics_sdr(omega, z, beta, projections[i], n)
    unitary_constr_real, unitary_constr_imag = api.unitary_constr(N_omega, N_z, projections[i])
    constr_sympl += sympl_real + sympl_imag
    constr_dyn += dyn_real_plus + dyn_imag_plus + dyn_real_minus + dyn_imag_minus
    constr_unitary += [unitary_constr_real + unitary_constr_imag]

In [5]:
# Summing all constraints together
constraints_list = constr_sympl + constr_dyn + constr_unitary

In [6]:
# Objective function
obj_f = api.obj_f_sdr(N_omega,N_z)

In [7]:
[np.trace(Y.conj().T@constr_sympl[i]@Y) for i in range(len(constr_sympl)) if np.abs(np.trace(Y.conj().T@constr_sympl[i]@Y)) >= 10**-6]

[]

In [8]:
[np.trace(Y.conj().T@constraints_list[i]@Y) for i in range(len(constraints_list)) if np.trace(Y.conj().T@constraints_list[i]@Y) > 10**-5]

[]

In [9]:
# CVXPY model
variable = cp.Variable(shape = ((2*N_z + 2)*N_omega,(2*N_z + 2)*N_omega), hermitian = True)
constraints = [variable >> 0]
# Constraint on symplectic structure of propagators
constraints += [cp.real(cp.trace(constr_sympl[i]@variable)) == 0 for i in range(len(constr_sympl))]
# Constraint on dynamics
constraints += [cp.real(cp.trace(constr_dyn[i]@variable)) == 0 for i in range(len(constr_dyn))]
# Constraint that the top matrix is unitary
constraints += [cp.real(cp.trace(constr_unitary[i]@variable)) == 0 for i in range(len(constr_unitary))]
# Constraint that defines the problem into a semidefinite relaxation
constraints += [cp.real(cp.trace(sdr_def_constr[i]@variable)) == sdr_cst[i] for i in range(len(sdr_def_constr))]
# Constraint that says the mean photon number at next step is bigger than previous
constraints += [cp.real(cp.trace(photon_nbr_prev[i]@variable)) >= 0 for i in range(len(photon_nbr_prev))]
# Constraint that defines the photon number at end of waveguide
constraints.append(cp.real(cp.trace(photon_end@variable)) == 0)
problem = cp.Problem(cp.Maximize(cp.real(cp.trace(obj_f@variable))), constraints)

In [13]:
# seems to converge to negative of sum of singular values?
problem.solve(verbose = True, eps_abs=1e-9, eps_rel=1e-9, max_iters = 5*10**5)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Mar 11 02:34:01 PM: Your problem has 4356 variables, 1697 constraints, and 0 parameters.
(CVXPY) Mar 11 02:34:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 11 02:34:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 11 02:34:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 11 02:34:01 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 11 02:34:01 PM: Using cached ASA map, for faster compil

3.814332476591024

In [14]:
optimal_variable = variable.value

In [40]:
np.save("explicit_fixed_pump_max.npy", optimal_variable)

In [15]:
np.trace(obj_f@optimal_variable)

(3.814332476591024+5.421010862427522e-20j)

In [16]:
# How much every constraints are violated
print("Symplectic constraints violated: ", [np.trace(constr_sympl[i]@optimal_variable) for i in range(len(constr_sympl)) if np.abs(np.trace(constr_sympl[i]@optimal_variable)) >= 5*10**-4])
print("Dynamics constraints violated: ", [np.trace(constr_dyn[i]@optimal_variable) for i in range(len(constr_dyn)) if np.abs(np.trace(constr_dyn[i]@optimal_variable)) >= 5*10**-4])
print("Unitary constraints violated: ", [np.trace(constr_unitary[i]@optimal_variable) for i in range(len(constr_unitary)) if np.abs(np.trace(constr_unitary[i]@optimal_variable)) >= 5*10**-4])
print("SDR definitiion constraints violated: ", [np.trace(sdr_def_constr[i]@optimal_variable) for i in range(len(sdr_def_constr)) if np.abs(np.trace(sdr_def_constr[i]@optimal_variable)) - sdr_cst[i] >= 2*10**-5])
print("Symplectic constraints violated: ", [np.trace(photon_nbr_prev[i]@optimal_variable) for i in range(len(photon_nbr_prev)) if (np.trace(photon_nbr_prev[i]@optimal_variable)) <= 10**-5])

Symplectic constraints violated:  []
Dynamics constraints violated:  []
Unitary constraints violated:  []
SDR definitiion constraints violated:  []
Symplectic constraints violated:  []


In [19]:
[np.trace(optimal_variable@constraints_list[i]) for i in range(len(constraints_list)) if np.abs(np.trace(optimal_variable@constraints_list[i])) >= 10**-3]

[]

In [20]:
S, V, D = scipy.linalg.svd(W_plus[-1] - W_minus[-1])

In [22]:
np.sum(V)

3.5333138127727044